<img src="Electronic_Brain.png" width="200" style="float:left">
<h1> Summer 2021 ML Course.</h1>
<h2> Exercise 7: Nonlinear Regression Models, Distribution Drift<br>Tools: Numpy, Pandas, Scikit-Learn</h2>

In [ ]:
!python --version
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import gauss, randint
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from icecream import ic

In [ ]:
# Set a few defaults.
np.random.seed(42)
pd.set_option('display.precision', 3)
plt.rcParams['figure.figsize'] = [15, 5]

<img src="desktop-computer-icon.png" width="80" style="float:left">
<h1> &nbsp; Section A: Signal Generation.</h1>

In [ ]:
"""
Generate AAPL's "bipolar" behavior, alternating between price levels of 10 and 30 USD with noise, for 20 days at a time.
Our target function today is a "long straddle".
Basically, the long straddle makes money on big price swings (up *OR* down),
and loses when prices are "calm" (within a relatively flat range).
"""
def generate_prices(sigma=5, regime_len=20, num_regimes=20):
    prices_df = pd.DataFrame()
    appl_prices = np.array([])
    for i in range(num_regimes):
        curr_prices = 20 + 10*(-1)**i + np.random.normal(0, sigma, regime_len)
        appl_prices = np.append(appl_prices, curr_prices)

    prices_df['AAPL'] = appl_prices

    # Generate backward & forward 10-day gains (*NOT* rolling means).
    prices_df['AAPL_back_10_day_gain'] = prices_df['AAPL'] - prices_df['AAPL'].shift(10)
    prices_df['AAPL_back_30_day_gain'] = prices_df['AAPL'] - prices_df['AAPL'].shift(30)

    # Create the "long straddle" (continuous) target function and its discrete version.
    prices_df['target'] = abs(prices_df['AAPL'] - prices_df['AAPL'].shift(-10)) - 20
    prices_df['discrete_target'] = prices_df['target'] > 0

    # Return a "clean" version of the prices dataframe, with no missing values.
    prices_df.dropna(inplace=True)
    return(prices_df)

In [ ]:
Apple_df = generate_prices()
X = Apple_df[['AAPL_back_10_day_gain', 'AAPL_back_30_day_gain']]
y = Apple_df['target']
y_discrete = Apple_df['target'] > 0

In [ ]:
fig, ax = plt.subplots()
ax.plot(Apple_df['AAPL'], label='AAPL')
ax.set(xlabel='Day Seq. Number', title='AAPL Prices');
plt.legend();

<img src="desktop-computer-icon.png" width="80" style="float:left">
<h1> &nbsp; Section B: Feature & Target Extraction and Visualization</h1>

In [ ]:
plt.rcParams['figure.figsize'] = [8,6]
Apple_df.plot.scatter(x="AAPL_back_10_day_gain", y="AAPL_back_30_day_gain", c="target", cmap='jet', title='Forward gain');

In [ ]:
"""
Show the linear regression's output on the 2D plane defined by the two features.
"""
def show_regression(reg):
    # Define bounds of the domain.
    min1, max1 = Apple_df['AAPL_back_10_day_gain'].min()-5, Apple_df['AAPL_back_10_day_gain'].max()+5
    min2, max2 = Apple_df['AAPL_back_30_day_gain'].min()-5, Apple_df['AAPL_back_30_day_gain'].max()+5

    # Define the x and y grid.
    x1grid = np.linspace(min1, max1, 100)
    x2grid = np.linspace(min2, max2, 100)

    # Create the 2D grid.
    xx, yy = np.meshgrid(x1grid, x2grid)

    # Flatten each grid to a vector
    r1, r2 = xx.flatten(), yy.flatten()
    r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))

    # Horizontal stack vectors to create x1,x2 input for the model
    grid = np.hstack((r1,r2))

    # Use the model to make predictions on the grid.
    y_hat = reg.predict(grid) # For all real-valued regression algs.

    # Reshape the predictions back into a grid.
    zz = y_hat.reshape(xx.shape)

    # Plot the grid of x, y and z values as a surface
    plt.rcParams['figure.figsize'] = [8,6]
    plt.contourf(xx, yy, zz, cmap='gray')

    # Add the actual training set as a scatter plot.
    c = plt.scatter(x=Apple_df["AAPL_back_10_day_gain"],
                    y=Apple_df["AAPL_back_30_day_gain"],
                    c=Apple_df["target"], cmap='jet');
    # Add a color bar.
    plt.colorbar(c);

In [ ]:
"""
Perform 100-fold cross validation to estimate the regressor's accuracy.
Check the scikit-learn documentation to check how score() scores the regression algorithms.
"""
def eval_regression(reg_func, target_field, X_df):
    score_sum=0
    for _ in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X_df[["AAPL_back_10_day_gain", "AAPL_back_30_day_gain"]], X_df[target_field], test_size=0.1)
        reg_func.fit(X_train, y_train)
        score_sum += reg_func.score(X_test, y_test)
    return(score_sum/100)

<img src="desktop-computer-icon.png" width="80" style="float:left">
<h1> &nbsp; Section C: Regression Training and Visualization</h1>

In [ ]:
# Train a linear regressor on the continuous target.
ols_reg = LinearRegression()
ols_reg.fit(X, y)
# Show & evaluate the result.
show_regression(ols_reg)
score = eval_regression(ols_reg, 'target', Apple_df)
ic("Ordinary Least Squares score:", score);

<img src="Roni.png" width="50" style="float:left">
 &nbsp; Please consult the scikit-learn documentation and look up the score() function. <br>
 &nbsp; Then, explain to Ronny why his linear model failed to capitalize on Apple's predictable behavior...

In [ ]:
# Train K-nearest-neighbor regression.
num_neighbs=1
knn_reg = KNeighborsRegressor(n_neighbors=num_neighbs)
knn_reg.fit(X, y)
show_regression(knn_reg)
score = eval_regression(knn_reg, 'target', Apple_df)
print("KNN score for", num_neighbs, "neighbors:", score);

<img src="Motti.png" width="50" style="float:left">
 &nbsp; Kindly help Motti (manually) optimize the number of neighbors to achieve a better fit!

In [ ]:
# Try using ordinary least squares with polynomial features.
poly_order = 2
regressor = LinearRegression()
ols_poly_model = make_pipeline(PolynomialFeatures(poly_order), regressor)
ols_poly_model.fit(X,y)
show_regression(ols_poly_model)
score = eval_regression(ols_poly_model, 'target', Apple_df)
print("OLS regression with order", poly_order, "polynomial features:", score)

<img src="Batya.png" width="60" style="float:left">
 &nbsp; Kindly help Batya (manually) optimize the polynomial dimension to achieve a good fit!

In [ ]:
"""
Show the linear regression's output on the 2D plane defined by the two features.
This version is suitable for classifiers (with probability outputs) as opposed to regressors.
The only difference is the use of the (scikit-learn standard) predict_proba() function.
"""
def show_classification_probs(cls):
    # Define bounds of the domain.
    min1, max1 = Apple_df['AAPL_back_10_day_gain'].min()-5, Apple_df['AAPL_back_10_day_gain'].max()+5
    min2, max2 = Apple_df['AAPL_back_30_day_gain'].min()-5, Apple_df['AAPL_back_30_day_gain'].max()+5

    # Define the x and y grid.
    x1grid = np.linspace(min1, max1, 100)
    x2grid = np.linspace(min2, max2, 100)

    # Create the 2D grid.
    xx, yy = np.meshgrid(x1grid, x2grid)

    # Flatten each grid to a vector
    r1, r2 = xx.flatten(), yy.flatten()
    r1, r2 = r1.reshape((len(r1), 1)), r2.reshape((len(r2), 1))

    # Horizontal stack vectors to create x1,x2 input for the model
    grid = np.hstack((r1,r2))

    # Use the model to make predictions on the grid.
    y_hat = cls.predict_proba(grid) # Use predict_proba() to access classification algs' probabilities.

    # Reshape the predictions back into a grid.
    zz = y_hat[:,0].reshape(xx.shape) # Use the first class' probability.

    # Plot the grid of x, y and z values as a surface
    plt.rcParams['figure.figsize'] = [8,6]
    plt.contourf(xx, yy, zz, cmap='gray')

    # Add the actual training set as a scatter plot.
    c = plt.scatter(x=Apple_df["AAPL_back_10_day_gain"],
                    y=Apple_df["AAPL_back_30_day_gain"],
                    c=Apple_df["target"], cmap='jet');
    # Add a color bar.
    plt.colorbar(c);

In [ ]:
# Set up logistic regression for the problem.
poly_order = 2
regressor = LogisticRegression(random_state=0, max_iter=1500)
logistic_poly_model = make_pipeline(PolynomialFeatures(poly_order), regressor)
logistic_poly_model.fit(X,y_discrete)
show_classification_probs(logistic_poly_model)
score = eval_regression(logistic_poly_model, 'discrete_target', Apple_df)
print("Logistic regression:", score)

<img src="panicked_cat.png" width="120" style="float:left">
<h1> &nbsp; Section D: Re-evaluating Models on Drifting / Shifting Distributions</h1>
Please regenerate AAPL's price history, using values of regime_len in the [15, 60] range.<br>
Then, proceed to help our nervous cat check the model's performance when using the previously constructed OLS polynomial model on the new data (for each regime_len).

In [ ]:
# Regenerate AAPL's behavior, using different values of regime_len params.
# For each value, re-evaluate the *original* ols_poly_model on the new dataset, *without* retraining!
scores={}
for regime_len in range(15, 60):
    apple2_df = generate_prices(regime_len=regime_len)
    # -----------------------------------
    # Please fill in the function, placing
    # the values in the scores dictionary.
    # -----------------------------------

In [ ]:
fig, ax = plt.subplots()
ax.plot(scores.keys(), scores.values(), label='OLS_poly_model as function of regime_len')
ax.set(xlabel='regime_len', title='model score');
plt.legend();

Please explain the resulting graph to our cat. When should the cat panic?